In [ ]:
from openpyxl import *
from tkinter import *
import tkinter as tk
from PIL import Image,ImageTk
import os
import pyttsx3
from pyttsx3 import voice
import speech_recognition as sr
import sounddevice as sd
from scipy.io.wavfile import write
import soundfile as sf
from pyAudioAnalysis import audioSegmentation as aS
from datetime import date

In [2]:
import nltk
#It contains text processing libraries for tokenization, 
#parsing, classification, stemming, tagging and semantic reasoning.
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
#import curses
import string
import numpy

In [ ]:
import numpy as np
import tflearn

#TFlearn is a modular and transparent deep learning library built on top of Tensorflow. 
#It was designed to provide a higher-level API to TensorFlow 
#in order to facilitate and speed-up experimentations, while remaining fully transparent and compatible with it.
import tensorflow as tf
import random
tf.logging.set_verbosity(tf.logging.ERROR)

# import our chat-bot intents file
import json
#open the exact location   of CORPUS 
with open('docman.json') as jd:
    intents = json.load(jd)

In [ ]:
import speech_recognition as sr
r=sr.Recognizer()
# importing the pyttsx library
import pyttsx3
# initialisation 
engine = pyttsx3.init() 
engine.setProperty('rate', 160)
voices=engine.getProperty('voices')
engine.setProperty('voice', voices[1].id)
def bot_speaking(message): 
    # testing 
    engine.say(message)
    engine.runAndWait() 
def get_input():
    with sr.Microphone() as source:
        #print("Say something!!!");
        output.config(text="Listening...")
        bot_speaking("Speak now")
        
        audio=r.listen(source,timeout=0)
        #print("Perfect, Thanks!")
        bot_speaking("Perfect")
    try:
        msg=r.recognize_google(audio)
        output.config(text="YOU: "+msg); #r.recognize(audio,language='hi-IN')
        #bot_speaking("you said "+msg)
        return msg
    except:
        #print("Dude it's not working :(")
        bot_speaking("Sorry mate  check your Internet")
        output.config(tetx="WARNING : Check your Internet and Try again!!")
        pass;

In [ ]:
nltk.download('punkt')
#Punkt Sentence Tokenizer. This tokenizer divides a text into a list of sentences,
#by using an unsupervised algorithm to build a model for abbreviation words, collocations, and words that start sentences.
#It must be trained on a large collection of plaintext in the target language before it can be used.
#Create words, classes and documents
words = []
classes = []
documents = []
ignore_words = ['?','!']

In [ ]:
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']: # PATTERN IS USER QUESTION
      
        # tokenize each word in the sentence
        
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus        # add to our classes list
        if intent['tag'] not in classes:

            documents.append((w, intent['tag']))
            classes.append(intent['tag'])


In [ ]:
words = sorted(list(set([stemmer.stem(w.lower()) for w in words if w not in ignore_words])))

# remove duplicates
classes = sorted(list(set(classes)))


In [ ]:
print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

In [ ]:
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

In [ ]:
for doc in documents:
    # initialize our bag of words
    bag = []    #bag of words which is a representaion of text that describes the occurence of wordss
    #with in a doccument
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words: #if the word appears in the pattern put 1 else 0 
        bag.append(1) if w in pattern_words else bag.append(0)
    print(bag)
    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

In [ ]:
random.shuffle(training)
training = np.array(training)
print(training)


In [ ]:
train_x = list(training[:,0])
train_y = list(training[:,1])
print(train_x,train_y)


In [ ]:
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 20)
net = tflearn.fully_connected(net, 20)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs') 
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=2000, batch_size=20, show_metric=True)#n_epoch is the number of times network sees the data
model.save('model.tflearn')

In [ ]:
import pickle
pickle.dump({'words':words, 'classes':classes,'train_x':train_x,'train_y':train_y},open( "training_data", "wb" ))



# restore all of our data structures
import pickle
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

In [ ]:

import json
with open('docman.json') as jd:
    intents = json.load(jd)
    
# load our saved model
model.load('./model.tflearn')

def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [ ]:
# import openpyxl and tkinter modules 

r=sr.Recognizer()

# initialisation 
engine = pyttsx3.init() 
engine.setProperty('rate', 150)
voices=engine.getProperty('voices')
engine.setProperty('voice', voices[1].id)


def register():
    global register_screen
    register_screen = Toplevel(root)
    #register_screen.configure(background="#c71435")
    register_screen.title("Register")
    register_screen.geometry("320x550+400+50")
 
    global username
    global password
    global name
    global age
    global height
    global weight
    global sex
    global contact
    global username_entry
    global password_entry
    global name_entry
    global age_entry
    global height_entry
    global weight_entry
    global sex_entry
    global contact_entry
    username = StringVar()
    password = StringVar()
    name = StringVar()
    age = StringVar()
    height = StringVar()
    weight = StringVar()
    sex = StringVar()
    contact = StringVar()
    Label(register_screen, text="").pack()
    Label(register_screen, text="Please enter the details below", font="bold 15").pack()
    Label(register_screen, text="").pack()
    username_lable = Label(register_screen, text="Email id * ")
    username_lable.pack()
    username_entry = Entry(register_screen, textvariable=username)
    username_entry.pack()
    password_lable = Label(register_screen, text="Password * ")
    password_lable.pack()
    password_entry = Entry(register_screen, textvariable=password, show='*')
    password_entry.pack()
    name_lable = Label(register_screen, text="Name  ")
    name_lable.pack()
    name_entry = Entry(register_screen, textvariable=name)
    name_entry.pack()
    age_lable = Label(register_screen, text="Age * ")
    age_lable.pack()
    age_entry = Entry(register_screen, textvariable=age)
    age_entry.pack()
    height_lable = Label(register_screen, text="Height (in cms) * ")
    height_lable.pack()
    height_entry = Entry(register_screen, textvariable=height)
    height_entry.pack()
    weight_lable = Label(register_screen, text="Weight (in kgs) * ")
    weight_lable.pack()
    weight_entry = Entry(register_screen, textvariable=weight)
    weight_entry.pack()
    sex_lable = Label(register_screen, text="Sex (M/F) * ")
    sex_lable.pack()
    sex_entry = Entry(register_screen, textvariable=sex)
    sex_entry.pack()
    contact_lable = Label(register_screen, text="Contact No. * ")
    contact_lable.pack()
    contact_entry = Entry(register_screen, textvariable=contact)
    contact_entry.pack()
    Label(register_screen, text="").pack()
    Button(register_screen, text="Register", width=10, height=1, bg="#cf2929", fg="white",command = register_user).pack()
    Label(register_screen,text="").pack()
    Button(register_screen, text="Back", width=10, height=1, bg="black", fg="white",command = delete_register_user).pack()

def delete_register_user():
    register_screen.destroy()
 
 
# Designing window for login 
 
def login():
    global login_screen
    login_screen = Toplevel(root)
    #login_screen.configure(background="white")
    login_screen.title("Login")
    login_screen.geometry("320x550+400+50")
    Label(login_screen, text="").pack()
    Label(login_screen, text="").pack()
    Label(login_screen, text="").pack()
    Label(login_screen, text="Please enter the details below to login",font="bold 12").pack()
    Label(login_screen, text="").pack()
 
    global username_verify
    global password_verify
 
    username_verify = StringVar()
    password_verify = StringVar()
    
 
    global username_login_entry
    global password_login_entry
 
    Label(login_screen, text="Email id * ").pack()
    username_login_entry = Entry(login_screen, textvariable=username_verify)
    username_login_entry.pack()
    Label(login_screen, text="").pack()
    Label(login_screen, text="Password * ").pack()
    password_login_entry = Entry(login_screen, textvariable=password_verify, show= '*')
    password_login_entry.pack()
    Label(login_screen, text="").pack()
    Button(login_screen, text="Login", width=10, height=1, command = login_verify).pack()

#Implementing event on register button
 
def register_user():
 
    username_info = username.get()
    password_info = password.get()
    current_id=0

    global name_info
    name_info= name.get()
    age_info = age.get()
    height_info = height.get()
    weight_info = weight.get()
    sex_info = sex.get()
    contact_info = contact.get()
 
    file = open(username_info, "w")
    file.write(username_info + "\n")
    file.write(password_info + "\n")
    file.write(name_info + "\n")
    file.write(age_info + "\n")
    file.write(height_info + "\n")
    file.write(weight_info + "\n")
    file.write(sex_info + "\n")
    file.write(contact_info + "\n")
    file.write(str(current_id) + "\n")


    file.close()
 
    username_entry.delete(0, END)
    password_entry.delete(0, END)
    name_entry.delete(0,END)
    age_entry.delete(0,END)
    height_entry.delete(0,END)
    weight_entry.delete(0,END)
    sex_entry.delete(0,END)
    contact_entry.delete(0,END)
 
    Label(register_screen, text="Registration Successful", fg="green", font=("calibri", 11,"bold")).pack()
    
# Implementing event on login button 
 
def login_verify():
    global username1
    username1 = username_verify.get()
    password1 = password_verify.get()
    username_login_entry.delete(0, END)
    password_login_entry.delete(0, END)
    global user
    user="friend"
    list_of_files = os.listdir()
    if username1 in list_of_files:
        file1 = open(username1, "r")
        verify = file1.read().splitlines()
        if password1 in verify:
            user=verify[2]
            login_sucess()
 
        else:
            password_not_recognised()
 
    else:
        user_not_found()
# Designing popup for login success
 
def login_sucess():
    global login_success_screen
    login_success_screen = Toplevel(login_screen)
    login_success_screen.title("Success")
    login_success_screen.geometry("150x100+480+270")
    Label(login_success_screen, text="Login Success!",fg="green",font="bold 13").pack()
    Button(login_success_screen, text="OK", command=delete_login_success).pack()
# Designing popup for login invalid password
 
def password_not_recognised():
    global password_not_recog_screen
    password_not_recog_screen = Toplevel(login_screen)
    password_not_recog_screen.title("Success")
    password_not_recog_screen.geometry("150x100+480+270")
    Label(password_not_recog_screen, text="Invalid Password",fg="red",font="bold 13").pack()
    Button(password_not_recog_screen, text="OK", command=delete_password_not_recognised).pack()
# Designing popup for user not found
 
def user_not_found():
    global user_not_found_screen
    user_not_found_screen = Toplevel(login_screen)
    user_not_found_screen.title("Success")
    user_not_found_screen.geometry("150x100+480+270")
    Label(user_not_found_screen, text="User Not Found!!",fg="red",font="bold 13").pack()
    Button(user_not_found_screen, text="OK", command=delete_user_not_found_screen).pack()
# Deleting popups
 
def delete_login_success():
    login_success_screen.destroy()
    login_screen.destroy()
    home_screen()
 
 
def delete_password_not_recognised():
    password_not_recog_screen.destroy()
 
 
def delete_user_not_found_screen():
    user_not_found_screen.destroy()
    
def home_screen():
    global home_screen
    home_screen = Toplevel(root)
    home_screen.title("HOME")
    home_screen.geometry("320x550+400+50")
    home_screen.configure(background='white')
    Label(home_screen,text="Welcome", bg="white",fg="#cf2929" ,width="300", height="2", font=("Calibri", 18)).pack()
    Label(home_screen,text="").pack()
    Button(home_screen, text="About", width=15, fg="white",height=1, bg="#cf2929",font="bold 12" ,command = about).pack()
    Label(home_screen,text="").pack()
    Button(home_screen, text="Details", width=15, fg="white",height=1, bg="#cf2929",font="bold 12" , command = record_snoring).pack()
    Label(home_screen,text="").pack()
    Button(home_screen, text="choose method", width=15, fg="white",height=1, bg="#cf2929",font="bold 12" , command = health_journal).pack()
    Label(home_screen,text="").pack()
    Button(home_screen, text="Queue", width=15, fg="white",height=1, bg="#cf2929",font="bold 12" , command = chatbot).pack()
    Label(home_screen,text="").pack()
    Button(home_screen, text="Enter Patient Name", width=15, fg="white",height=1, bg="#cf2929", font="bold 12" ,command = reports).pack()
    Label(home_screen,text="").pack()
    Button(home_screen, text="Instructions", width=15, fg="white",height=1, bg="#cf2929",font="bold 12" , command = instructions).pack()
    Label(home_screen,text="").pack()
    Button(home_screen, text="Contact", width=15, fg="white",height=1, bg="#cf2929", font="bold 12" ,command = contact).pack()
    Label(home_screen,text="").pack()
    Button(home_screen, text="Log Out", width=15, fg="white",height=1, bg="black", font="bold 12" ,command = delete_home_screen).pack()
    Label(home_screen,text="").pack()
    
def delete_home_screen():
    home_screen.destroy()
    #root.destroy()
    

    
def bot_speaking(message): 
    # testing 
    #rate = engine.getProperty('rate')   # getting details of current speaking rate
    #print (rate)
    engine.say(message)
    engine.runAndWait() 
    
def about():
    global about_screen
    about_screen = Toplevel(home_screen)
    about_screen.title("ABOUT")
    about_screen.geometry("320x550+400+50")
    about_screen.configure(background='white')
    Label(about_screen,text="DocMan", bg="white",fg="#c71435" ,width="300", height="2", font=("Calibri", 18)).pack()
    Label(about_screen,text="").pack()
    
    '''img1 = Image.open("R:\\PROJECTS\\DocMan -- Snoring Analysis\\bg.png") 
    img = img1.resize((500, 300), Image.ANTIALIAS)
    photo1=ImageTk.PhotoImage(img)
    lab=Label(about_screen,image=photo1).pack()#.place(x=200,y=30)'''
    description="Welcome to DocMan!"+"\n"+"If you are a person who is suffering with snoring, then you’ve come to the right place at the right time."+"\n"+"Sleep illnesses are often left undiagnosed as their symptoms go unnoticed, unless you have a sleeping partner."+"\n"+"Snoring is not a merely a sound produced due to nose block, there are numerous other reasons behind a person’s snoring."+"\n"+" Some can be extremely malicious to our body, if left unchecked it may lead to a much horrifying doom called death."+"\n"+" Hence, it has become a necessity to know the underlying cause of snoring."+"\n"+"People who complain of snoring are often diagnosed with Obstructive Sleep Apnea."+"\n"+" Obstructive sleep apnea (also called as OSA) is a condition which causes pauses in breathing while the person is sleeping."+"\n"+"This leads to lack of oxygen in the body."+"\n"+"DocMan app helps you record your snoring and analyses it and determines if you are suffering with OSA."+"\n"+" If you are suffering with OSA it also tells us if OSA is mild, moderate or severe."+"\n"+"If you are a new user please visit instructions tab provided on home page."+"\n"+"Thank you."
    #Label(about_screen,text=description,width="150", height="14", font=("Calibri", 12)).pack()
    scrollbar = Scrollbar(about_screen, orient='horizontal',width='5')
    scrollbar.pack(fill=X)
    listbox = Listbox(about_screen,xscrollcommand=scrollbar.set,width='90',height='20')
    listbox.insert(END,"")
    listbox.insert(END,"")
    listbox.insert(END,"  Welcome to DocMan!")
    listbox.insert(END,"  If you are a person who is suffering with snoring, then you’ve come to the right place at the right time.")
    listbox.insert(END,"  Sleep illnesses are often left undiagnosed as their symptoms go unnoticed, unless you have a sleeping partner.")
    listbox.insert(END,"  Snoring is not a merely a sound produced due to nose block, there are numerous other reasons behind a person’s snoring.")
    listbox.insert(END,"  People who complain of snoring are often diagnosed with Obstructive Sleep Apnea.")
    listbox.insert(END,"  Obstructive sleep apnea (also called as OSA) is a condition which causes pauses in breathing while the person is sleeping.")
    listbox.insert(END,"  This leads to lack of oxygen in the body.")
    listbox.insert(END,"  DocMan app helps you record your snoring and analyses it and determines if you are suffering with OSA.")
    listbox.insert(END,"  If you are suffering with OSA it also tells us if OSA is mild, moderate or severe.")
    listbox.insert(END,"  If you are a new user please visit instructions tab provided on home page.")
    listbox.insert(END,"  Thank you.")
    listbox.pack( fill=BOTH)
    Label(about_screen,text="").pack()
    Label(about_screen,text="").pack()
    scrollbar.config(command=listbox.xview)
    Button(about_screen,text="Listen", height="1", width="10",fg="white", bg="#cf2929", command = speak_about).pack()
    Label(about_screen,text="").pack()
    Button(about_screen, text="Back", width=10, height=1, bg="black", fg="white",command = delete_about_screen).pack()

def delete_about_screen():
    about_screen.destroy()
    
    
def speak_about():
    
    '''list_of_files = os.listdir()
    if username1 in list_of_files:
        file1 = open(username1, "r")
        verify = file1.read().splitlines()
        global user
        user=verify[2]
        #print(user)'''
    bot_speaking("hello"+user)
    bot_speaking("I am your personal assistant honey")
    bot_speaking("I will walk you through the process of using DocMan")
    bot_speaking("If you are a new user please visit instructions tab provided on home page")
    bot_speaking("Thank you")
    
    


def record_snoring():
    file1 = open(username1, "r")
    verify = file1.read().splitlines()
    current_id=int(verify[-1])
    current_id+=1
    file1.close()
    file1=open(username1,"a")
    file1.write(str(current_id) + "\n")
    
    file1.close()
    if current_id%6==0:
        current_id=0
    global record_screen
    record_screen = Toplevel(home_screen)
    record_screen.title("Recording result")
    record_screen.geometry("320x550+400+50")
    record_screen.configure(background='white')
    Label(record_screen,text="").pack()
    Label(record_screen,text="").pack()
    Label(record_screen,text="").pack()
    Label(record_screen,text="").pack()
    Label(record_screen,text="Recorded Successfully",width="300", height="1",fg="green" ,font=("Calibri", 12,"bold")).pack()
    Label(record_screen,text="").pack()
    Label(record_screen,text="Current snoring signal id : DM_"+str(current_id),width="300", height="1",fg="black" ,font=("Calibri", 10,"bold")).pack()
    bot_speaking("Recording will be started in three seconds")
    fs = 44100  # Sample rate
    seconds = 5  # Duration of recording

    myrecording = sd.rec(int(seconds * fs), samplerate=fs, channels=2)
    sd.wait()  # Wait until recording is finished
    fname=user+"DM_"+str(current_id)+".wav"
    write(fname, fs, myrecording)
    bot_speaking("Recorded Succesfuly")
    bot_speaking("Please visit reports provided on home page for results.")
    Label(record_screen,text="").pack()
    Label(record_screen,text="").pack()
    Button(record_screen, text="Back", width=10, height=1, bg="black", fg="white",command = delete_record_screen).pack()

def delete_record_screen():
    record_screen.destroy()


    
    
    

def reports():
    global report_screen
    report_screen = Toplevel(home_screen)
    report_screen.title("Reports")
    report_screen.geometry("320x550+400+50")
    report_screen.configure(background='white')
    Label(report_screen,text="Report",width="300", height="2",fg="#cf2929" ,font=("Calibri", 12,"bold")).pack()
    Label(report_screen,text="").pack()
    bot_speaking("Please wait for sometime Your result will be displayed soon")
    #audio_path=user+"snoring.wav"
    audio_path="00000176.wav"
    #y,sr=sf.read(audio_path,dtype='float32')
    #y=y.flatten()
    n_speakers=10
    '''filename=audio_path
    filename2=r"R:\PROJECTS\DocMan -- Snoring Analysis\Data\21119-00000338\00000338.wav"
    filename3=r"R:\PROJECTS\DocMan -- Snoring Analysis\Data\22186-00000037\00000037.wav"'''
    sam=aS.speakerDiarization(audio_path, n_speakers, mt_size=2.0, mt_step=0.2,st_win=0.05, lda_dim=10, plot_res=False)
    y,sr=sf.read(audio_path,dtype='float32')
    y=y.flatten()
    duration=int(len(y)/sr)
    newsr=int((len(sam)/len(y))*sr)
    look0='0'*(newsr*10)
    #look1='1'*(newsr*10)
    s=""
    for i in range(len(sam)):
        s+=str(int(sam[i]))
    p=s.count(look0)
    #q=s.count(look1)
    ahi_0=3600*p/(len(y)/sr)
    AHI=ahi_0
    global out
    out=""
    if AHI<5:
        out="normal"
        #print("RESULT : NORMAL(NO SLEEP APNEA)")
    elif AHI>=5 and AHI<15:
        out="mild"
        #print("RESULT : MILD SLEEP APNEA")
    elif AHI>=15 and AHI<35:
        out="moderate"
        #print("RESULT : MODERATE SLEEP APNEA")
    elif AHI>=30:
        out="severe"
        #print("RESULT : SEVERE SLEEP APNEA")
    Label(report_screen,text="You are in "+"Moderate"+" stage",width="300", height="2",fg="black", font=("Calibri", 14)).pack()
    Label(report_screen,text="").pack()
    Button(report_screen,text="Listen", height="1", width="20",fg="white", bg="#cf2929", command = speak_report).pack()
    Label(report_screen,text="").pack()
    Button(report_screen, text="Back", width=10, height=1, bg="black", fg="white",command = delete_report_screen).pack()

def speak_report():
    bot_speaking("hey"+user)
    bot_speaking("you are in "+"Moderate"+" stage")
    bot_speaking("dont worry we suggest you some precautions")
    
def delete_report_screen():
    report_screen.destroy()
 
    
    

def instructions():
    global instructions_screen
    instructions_screen = Toplevel(home_screen)
    instructions_screen.title("Instructions")
    instructions_screen.geometry("320x550+400+50")
    instructions_screen.configure(background='white')
    Label(instructions_screen,text="Instructions", bg="white",fg="#c71435" ,width="300", height="2", font=("Calibri", 18)).pack()
    Label(instructions_screen,text="").pack()
    Label(instructions_screen,text="").pack()
    #description="Welcome to DocMan!"+"\n"+"If you are a person who is suffering with snoring, then you’ve come to the right place at the right time."+"\n"+"Sleep illnesses are often left undiagnosed as their symptoms go unnoticed, unless you have a sleeping partner."+"\n"+"Snoring is not a merely a sound produced due to nose block, there are numerous other reasons behind a person’s snoring."+"\n"+" Some can be extremely malicious to our body, if left unchecked it may lead to a much horrifying doom called death."+"\n"+" Hence, it has become a necessity to know the underlying cause of snoring."+"\n"+"People who complain of snoring are often diagnosed with Obstructive Sleep Apnea."+"\n"+" Obstructive sleep apnea (also called as OSA) is a condition which causes pauses in breathing while the person is sleeping."+"\n"+"This leads to lack of oxygen in the body."+"\n"+"DocMan app helps you record your snoring and analyses it and determines if you are suffering with OSA."+"\n"+" If you are suffering with OSA it also tells us if OSA is mild, moderate or severe."+"\n"+"If you are a new user please visit instructions tab provided on home page."+"\n"+"Thank you."
    description="Keep your phone within a radius less than one meter."+"\n"+"Record your snoring signal."+"\n"+"After recording the Snoring signal visit reports page."+"\n"+"Results will be shown."+"\n"+"For precautions use our chat assistance."
    

    listbox = Listbox(instructions_screen,width='80',height='15')
    '''for i in range(1000):
        listbox.insert(END, str(i))'''
    listbox.insert(END,"")
    listbox.insert(END,"")
    listbox.insert(END,"  Keep your phone within a radius less than one meter.")
    listbox.insert(END,"  Record your snoring signal.")
    listbox.insert(END,"  After recording the Snoring signal visit reports page.")
    listbox.insert(END,"  Results will be shown.")
    listbox.insert(END,"  For precautions use our chat assistance.")
    listbox.pack( fill=BOTH)
    Label(instructions_screen,text="").pack()
    Button(instructions_screen,text="Listen", height="1", width="10",fg="white", bg="#cf2929", command = speak_instructions).pack()
    Label(instructions_screen,text="").pack()
    Button(instructions_screen, text="Back", width="10", height="1", bg="black", fg="white",command = delete_instructions_screen).pack()
    #scrollbar.config(command=listbox.yview)
    '''Label(instructions_screen,text=description,width="150", height="14", font=("Calibri", 12)).pack()
    Label(instructions_screen,text="").pack()'''


def speak_instructions():
    bot_speaking("hello"+user)
    bot_speaking("Instructions to use DocMan")
    bot_speaking("Keep your phone within a radius less than one meter")
    bot_speaking("record your snoring signal")
    bot_speaking("After recording the Snoring signal visit reports page")
    bot_speaking("results will be shown")
    bot_speaking("for precautions use our chat assistance")
    
    
def delete_instructions_screen():
    instructions_screen.destroy()
    
    
def contact():
    global contact_screen
    contact_screen = Toplevel(home_screen)
    contact_screen.title("Contact")
    contact_screen.geometry("320x550+400+50")
    contact_screen.configure(background='white')
    Label(contact_screen,text="Contact", bg="white",fg="black" ,width="300", height="1", font=("Calibri", 14)).pack()
    Label(contact_screen,text="").pack()
    Label(contact_screen,text="Mail us at: docman.healthcare@gmail.com", bg="white",fg="black" ,width="300", height="1", font=("Calibri", 12)).pack()
    Label(contact_screen,text="").pack()
    Button(contact_screen, text="Back", width=10, height=1, bg="black", fg="white",command = delete_contact_screen).pack()

def delete_contact_screen():
    contact_screen.destroy()
    
    
def health_journal():
    global var1
    global var2
    global var3
    global a
    global b
    global c
    global today
    global journal_screen
    
    journal_screen = Toplevel(home_screen)
    journal_screen.title("Health Journal")
    journal_screen.geometry("320x550+400+50")
    journal_screen.configure(background='white')
    Label(journal_screen,text="Health Journal",width="300", height="2",fg="#cf2929" ,font=("Calibri", 18,"bold")).pack()
    Label(journal_screen,text="").pack()
    Label(journal_screen,text="1.Did you experience a sore or dirty",width="300", height="1",font=("Calibri", 9)).pack()
    Label(journal_screen,text="throat this morning?(YES/NO)", height="1",font=("Calibri", 9)).pack()
    Label(journal_screen,text="").pack()
    today = str(date.today())
    var1=StringVar()
    a=Entry(journal_screen, textvariable=var1)
    a.pack()
    Label(journal_screen,text="").pack()
    Label(journal_screen,text="2.Were you feeling drowsy and tried",width="300", height="1",font=("Calibri", 9)).pack()
    Label(journal_screen,text="throughout the day?(YES/NO)", height="1",font=("Calibri", 9)).pack()
    Label(journal_screen,text="").pack()
    
    
    var2=StringVar()
    b=Entry(journal_screen, textvariable=var2)
    b.pack()
    Label(journal_screen,text="").pack()
    Label(journal_screen,text="3.Tell us any other symptoms", height="1",font=("Calibri", 9)).pack()
    Label(journal_screen,text="").pack()
    var3=StringVar()
    c=Entry(journal_screen, textvariable=var3)
    c.pack()
    Label(journal_screen,text="").pack()
    Label(journal_screen,text="").pack()
    Label(journal_screen,text="").pack()
    Button(journal_screen, text="Submit", width=10, height=1, bg="green", fg="white",command = journal_submit_screen).pack()
    Label(journal_screen,text="").pack()
    Label(journal_screen,text="").pack()
    Button(journal_screen, text="Back", width=10, height=1, bg="black", fg="white",command = delete_journal_screen).pack()

def delete_journal_screen():
    journal_screen.destroy()
def journal_submit_screen():
    resp1=var1.get()
    resp2=var2.get()
    resp3=var3.get()
    file = open(username1+" journal", "w")
    file.write("Health Jounral Responses "+ today+"\n")
    
    a.delete(0,END)
    b.delete(0,END)
    c.delete(0,END)
 
    Label(journal_screen, text="Success!", fg="green", font=("calibri", 11,"bold")).pack()
    
        
    file.write('Fisrt respose : '+resp1 + "\n")
    file.write('Second respose : '+resp2 + "\n")
    file.write('Third respose : '+resp3 + "\n")
    file.close()
    
    '''scrollbar = Scrollbar(journal_screen)
    scrollbar.pack(side=RIGHT, fill=Y)

    listbox = Listbox(journal_screen, yscrollcommand=scrollbar.set)
    for i in range(1000):
        listbox.insert(END, str(i))
    listbox.pack(side=LEFT, fill=BOTH)

    scrollbar.config(command=listbox.yview)'''
    

#========================Chatbot=====================
'''def cb(event):
        user_text = user_input.get()
        if user_text in greetings:
            bot_text = random.choice(greetings)
        elif user_text in question:
            bot_text = random.choice(responses)
        else:
            bot_text = huh
        output.config(text=bot_text)
        bot_speaking(bot_text)'''
def cchat(event):
    output.config(text="BOT: I am  your Personal Digital Assistant. What can I do for you!")
    output.config(text="BOT : Hey mate. How can I help you?")
    #bot_speaking("Hey mate how can I help you?")
        #user_text = user_input.get()

    inp=user_input.get()  #input("YOU : ")
 
        
    results = model.predict([bow(inp, words)])[0]
    results_index=numpy.argmax(results)
    tag=classes[results_index]
    if(results[results_index]>0.65):
        for tg in intents['intents']:
            if tg['tag']==tag:
                response=tg['responses']
        ms=random.choice(response)

        output.config(text="BOT : " +ms)
        bot_speaking(ms)
    else:
        output.config(text="BOT :  I did not understand you! Try again")
        bot_speaking("I did not understand you! Try again")
    
def cb():
    Label(chatbot_screen,text="Honie speaking :D",width='20',height="1", font=("Calibri", 9)).pack()
    
    #Label(chatbot_screen,text="BOT : Hey mate. How can I help you?",height="1", font=("Calibri", 9)).pack()
    output.config(text="BOT: I am  your Personal Digital Assistant. What can I do for you!")
    output.config(text="BOT : Hey mate. How can I help you?")
    bot_speaking("Hey mate how can I help you?")
    
        #user_text = user_input.get()

    inp=str(get_input())   #input("YOU : ")
    
    results = model.predict([bow(inp, words)])[0]
    results_index=numpy.argmax(results)
    tag=classes[results_index]
    if(results[results_index]>0.65):
        for tg in intents['intents']:
            if tg['tag']==tag:
                response=tg['responses']
        ms=random.choice(response)

        output.config(text="BOT : " +ms)
        bot_speaking(ms)
    else:
        output.config(text="BOT :  I did not understand you! Try again")
        bot_speaking("I did not understand you! Try again")
    
def chatbot():
    global chatbot_screen
    chatbot_screen = Toplevel(home_screen)
    chatbot_screen.title("BOT")
    chatbot_screen.geometry("320x550+400+50")
    chatbot_screen.configure(background='white')
    img1  = Image.open("C:\\Users\\kumar\\a1.PROJECTS\\DocMan_MLR\\DocMan\\bg.png") 
    img = img1.resize((700, 250), Image.ANTIALIAS)
    photo=ImageTk.PhotoImage(img)
    lab=Label(image=photo).pack()
    Label(chatbot_screen,text="Chat Assistant", bg="white",fg="#cf2929" ,width="300", height="2", font=("Calibri", 18)).pack()
    Label(chatbot_screen,text="").pack()
    #.place(x=200,y=30)
    Label(chatbot_screen,text="Any queires? Ask me : ", bg="white",fg="black" ,width="200", height="1", font=("Calibri", 13)).pack()
    Label(chatbot_screen,text="").pack()
    
    global user_input
    global greetings
    global question
    global responses
    global huh
    global output
    
    
    user_input = tk.Entry(chatbot_screen)
    user_input.pack()
    
    #greetings = ['hola', 'hello', 'hi', 'Hi', 'hey!', 'hey']
    #question = ['How are you?', 'How are you doing?']
    #responses = ['Okay', "I'm fine"]
    huh = "I did not understand what you said"
    user_input.bind("<Return>", cchat)
    Label(chatbot_screen,text="").pack()
    Label(chatbot_screen,text="").pack()
    output = tk.Label(chatbot_screen, text='')
    output.pack()
    Label(chatbot_screen,text="").pack()
    Button(chatbot_screen, text="Speak with bot..", width=20, height=1, bg="black", fg="white",command = cb).pack()
    Label(chatbot_screen,text="").pack()
    Label(chatbot_screen,text="").pack()
    Label(chatbot_screen,text="").pack()
    Button(chatbot_screen, text="Back", width=10, height=1, bg="black", fg="white",command = delete_chatbot_screen).pack()

def delete_chatbot_screen():
    chatbot_screen.destroy()


def delete_root():
    root.destroy()

    
if __name__ == "__main__":
    global main_screen
    global root
    global current_id
    '''main_screen = Tk()
    main_screen.geometry("300x250")
    main_screen.title("Account Login")
    Label(text="Select Your Choice", bg="blue", width="300", height="2", font=("Calibri", 13)).pack()
    Label(text="").pack()
    Button(text="Login", height="2", width="30", command = login).pack()
    Label(text="").pack()
    Button(text="Register", height="2", width="30", command=register).pack()'''  
    # Driver code 
 
      
    # create a GUI window 
    root = Tk() 
    root.geometry("320x550+400+50")
    root.title("Smart Diagnostic Centre") 
      
    # set the background colour of GUI window 
    root.configure(background='white')
    img1  = Image.open("C:\\Users\\kumar\\a1.PROJECTS\\SCHEDULING ALGO\\IMAGES\\main.jpg") 
    img = img1.resize((700, 250), Image.ANTIALIAS)
    photo=ImageTk.PhotoImage(img)
    lab=Label(image=photo).pack()#.place(x=200,y=30)
    Label(text="Select Your Choice", bg="white", width="300", height="2", font=("Calibri", 13)).pack()
    Label(text="").pack()
    Button(text="Login", height="2", width="30",font="bold 10", command = login).pack()
    Label(text="").pack()
    Button(text="Register", height="2", width="30", font="bold 10",command=register).pack()
    Label(text="").pack()
    Label(text="").pack()
    Button(text="Close", height="2", width="30",bg="black",fg="white",font="bold 10",command=delete_root).pack()
  
    
    root.mainloop()
    
    #root=Tk()
    #root.mainloop() 